Tema 1:

Um foguete espacial, ao entrar na atmosfera, sofre deslocamentos d1, d2,..., dn em suas partes dadas pela
solução do sistema de equações lineares Ad = f, onde A é a matriz das propriedades, d é o vetor das incógnitas
e f é o vetor dos termos independentes (vetor constante). Caso o deslocamento de umas dessas partes passe de
2 cm o foguete explode, causando prejuízo gigantesco. Os deslocamentos podem ser negativos ou positivos,
dependendo da direção para onde ocorrem (dentro ou fora) e, por isso, **são considerados em módulo após os
cálculos efetuados**.

Uma das soluções possíveis para esse sistema é usar uma variação do método de fatoração
LU onde, se A é uma matriz não singular e A = LU, então A pode ser reescrita como A = LDP, onde L é a
mesma matriz triangular inferior do método convencional de fatoração LU, D é uma matriz diagonal e P é
uma matriz triangular superior com diagonal unitária. Desenvolva, então, um sistema para calcular os
deslocamentos d das partes de um foguete espacial considerado com todos os requisitos apresentados abaixo:

###a) Implementar algoritmo para calcular todos os deslocamentos {d} pelo método de Fatoração LU normal.

###b) Implementar algoritmo para calcular todos os deslocamentos {d} pelo método de Fatoração LDP então.

###c) Calibrar o sistema desenvolvido usando, como valores padrões, a matriz [A] e o vetor {f} dados abaixo.

###d) Fornecer um quadro resposta, para cada um dos métodos considerados, variando os valores de [A] e {f}.

###e) Analisar o que vai acontecer com o foguete espacial dessa questão, para o sistema mencionado abaixo.
\
[A] =
\begin{pmatrix}
3  & -2 &  1  \\
1  & -3 &  4  \\
9  &  4 & -5
\end{pmatrix}


{f}=
\begin{pmatrix}
8\\
6\\
11
\end{pmatrix}
Dados de entrada: n (número de deslocamentos), os termos de [A]nxn e os de {f}nx1.

Dados de saída: valores de {d}nx1 que representam os n deslocamentos d1, d2,..., dn.

###Definindo as matrizes e as bibliotecas

In [1]:
#bibliotecas
import math
import numpy as np

In [2]:
A = np.array([[3, -2, 1],
              [1, -3, 4],
              [9, 4, -5]], dtype=float)
B = np.array([[8],
              [6],
              [11]], dtype=float)
#print(A[0][1]) = Matriz A [linha][coluna]

###Transformando a Matriz em L e U e em L D P

---




In [3]:
def fatoracao_lu(matriz):
  n = matriz.shape[0]
  matrizUpper = matriz.astype(float) #copiando a matriz como float, essa será a U
  matrizLower = np.eye(n) #inicializando a matriz Identidade com o mesmo tamanho. essa será a L

  for i in range(n):
    for j in range(i+1,n):
      fator = matrizUpper[j][i] / matrizUpper[i][i] #calcula o fator multiplicativo
      matrizLower[j][i] = fator #armazena o valor multiplicativo

      for k in range(i,n):
        matrizUpper[j][k] -= fator * matrizUpper[i][k]

  return matrizLower, matrizUpper


In [4]:
def fatoracao_LDP(matriz):
  n = matriz.shape[0]
  matrizL, matrizU = fatoracao_lu(matriz)
  matrizD = np.zeros((n,n))
  matrizP = np.copy(matrizU)

  for i in range(n):
        if matrizU[i][i] == 0: #evitar divisões por zero
            raise ValueError(f"Divisão por zero na linha {i}")

        matrizD[i][i] = matrizU[i][i]  # diagonal principal somente
        for j in range(i, n):
            matrizP[i][j] = matrizU[i][j] / matrizD[i][i]  # diagonal principal igual a 1

  return matrizL, matrizD, matrizP

#####Testando as fatorações

In [5]:
#testando a fatoracao LU

L, U = fatoracao_lu(A)

# Multiplica L e U para verificar se o produto é igual a A
A_reconstruida = np.dot(L, U)

print("Matriz L (Triangular Inferior):") # Exibe as matrizes e o resultado da verificação
print(L)

print("\nMatriz U (Triangular Superior):")
print(U)

print("\nMatriz A refeita (L x U):")
print(A_reconstruida)

print("\nMatriz A original:")
print(A)

Matriz L (Triangular Inferior):
[[ 1.          0.          0.        ]
 [ 0.33333333  1.          0.        ]
 [ 3.         -4.28571429  1.        ]]

Matriz U (Triangular Superior):
[[ 3.         -2.          1.        ]
 [ 0.         -2.33333333  3.66666667]
 [ 0.          0.          7.71428571]]

Matriz A refeita (L x U):
[[ 3. -2.  1.]
 [ 1. -3.  4.]
 [ 9.  4. -5.]]

Matriz A original:
[[ 3. -2.  1.]
 [ 1. -3.  4.]
 [ 9.  4. -5.]]


In [6]:
#testando LDP

testeL, testeD, testeP = fatoracao_LDP(A)

LD = np.dot(testeL,testeD)
LDP = np.dot(LD, testeP)


print("matriz L\n", testeL)
print("matriz D\n", testeD)
print("matriz P\n", testeP)

print("matriz reconstruída\n", LDP)

print("matriz original\n", A)

matriz L
 [[ 1.          0.          0.        ]
 [ 0.33333333  1.          0.        ]
 [ 3.         -4.28571429  1.        ]]
matriz D
 [[ 3.          0.          0.        ]
 [ 0.         -2.33333333  0.        ]
 [ 0.          0.          7.71428571]]
matriz P
 [[ 1.         -0.66666667  0.33333333]
 [ 0.          1.         -1.57142857]
 [ 0.          0.          1.        ]]
matriz reconstruída
 [[ 3. -2.  1.]
 [ 1. -3.  4.]
 [ 9.  4. -5.]]
matriz original
 [[ 3. -2.  1.]
 [ 1. -3.  4.]
 [ 9.  4. -5.]]


###Resolvendo LDP e LU parte por parte


In [7]:
def substituicao_para_frente(L, b):
    n = L.shape[0]
    y = np.zeros(n)

    for i in range(n):
        soma = 0
        for j in range(i):
            soma += L[i, j] * y[j]
        y[i] = b[i] - soma

    return y

In [8]:
def substituicao_diagonal(D, y):
    n = D.shape[0]
    z = np.zeros(n)

    for i in range(n):
        z[i] = y[i] / D[i, i]  # D[i, i] é garantido ser um escalar

    return z

In [9]:
def substituicao_para_tras(U, z):
    n = U.shape[0]
    x = np.zeros(n)

    for i in reversed(range(n)):
        soma = 0
        for j in range(i+1, n):
            soma += U[i, j] * x[j]
        x[i] = (z[i] - soma) / U[i, i]

    return x

In [10]:
def resolver_LU(A, b):

  #Fatorar a matriz A em L e U
  matrizL, matrizU = fatoracao_lu(A)

  # Passo 1: Ly = b => achar o vetor y
  vetor_y = substituicao_para_frente(matrizL, b)

  # Passo 2: Resolver Ux = z (substituição para trás)
  vetor_solucao = substituicao_para_tras(matrizU, vetor_y)
  return vetor_solucao

In [11]:
def resolver_sistema_ldp(A, b):
    # Separar A em L, D e P
    L, D, U = fatoracao_LDP(A)

    # Passo 1: Resolver Ly = b (substituição para frente)
    y = substituicao_para_frente(L, b)

    # Passo 2: Resolver Dz = y (substituição diagonal)
    z = substituicao_diagonal(D, y)

    # Passo 3: Resolver Ux = z (substituição para trás)
    x = substituicao_para_tras(U, z)

    return x

###Recebendo uma matriz e um vetor qualquer

In [12]:
def receberMatriz():
  n = int(input("Digite a dimensão da matriz n x n:"))
  matrizInput = np.zeros((n,n))

  for i in range(n):
        for j in range(n):
            matrizInput[i][j] = float(input(f"Digite o elemento da posição ({i+1}, {j+1}): "))

  return matrizInput

In [13]:
def receberVetor():
  n = int(input("Digite o tamanho do vetor (deve ser igual ao da matriz):"))
  vetor_dados = np.zeros(n)
  for i in range(n):
    vetor_dados[i] = float(input(f"Digite o elemento da posição ({i+1})"))

  return vetor_dados

### Testes e aplicações


####Usando a função da biblioteca como parâmetro


In [14]:
#resolvendo com a biblioteca para ter um parâmetro
def solve_LU(L, U, B):


    L = np.array([L], dtype=float)
    U = np.array([U], dtype=float)
    B = np.array([B], dtype=float)

    y = np.linalg.solve (L, B)
    x = np.linalg.solve (U, y)
    return x

x = solve_LU (L, U, B)

print("Solução x:\n", x)

Solução x:
 [[[ 1.83333333]
  [-1.16666667]
  [ 0.16666667]]]


####Usando as funções implementadas

##### Para o A e B da questão

In [15]:
#resposta para LDP
ldp = resolver_sistema_ldp(A, B)
print("Solução para LDP:")
print(ldp)

#resposta para LU
LU = resolver_LU(A,B)
print("Solução para LU")
print(LU)

Solução para LDP:
[ 1.83333333 -1.16666667  0.16666667]
Solução para LU
[ 1.83333333 -1.16666667  0.16666667]


<ipython-input-7-921c08dd5286>:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[i] = b[i] - soma


#####Para A e B genéricos


In [16]:
matrizGenerica = receberMatriz()
vetorGererico = receberVetor()
print("Matriz inserida:\n",  matrizGenerica)
print("Vetor inserido:\n", vetorGererico)
#resposta para LDP
ldp = resolver_sistema_ldp(matrizGenerica, vetorGererico)
print("Solução para LDP:")
print(ldp)

#resposta para LU

LU = resolver_LU(matrizGenerica,vetorGererico)
print("Solução para LU")
print(LU)

Digite a dimensão da matriz n x n:3
Digite o elemento da posição (1, 1): 5
Digite o elemento da posição (1, 2): 3
Digite o elemento da posição (1, 3): 1
Digite o elemento da posição (2, 1): 5
Digite o elemento da posição (2, 2): 6
Digite o elemento da posição (2, 3): 1
Digite o elemento da posição (3, 1): 1
Digite o elemento da posição (3, 2): 6
Digite o elemento da posição (3, 3): 7
Digite o tamanho do vetor (deve ser igual ao da matriz):3
Digite o elemento da posição (1)1
Digite o elemento da posição (2)2
Digite o elemento da posição (3)3
Matriz inserida:
 [[5. 3. 1.]
 [5. 6. 1.]
 [1. 6. 7.]]
Vetor inserido:
 [1. 2. 3.]
Solução para LDP:
[-0.02941176  0.33333333  0.14705882]
Solução para LU
[-0.02941176  0.33333333  0.14705882]
